## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [614]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self._codigo = codigo
        self._nombre = nombre
        self._precio = precio
        self._tipo = tipo
        self._cantidad = cantidad

    @property
    def codigo(self):
        return self._codigo

    @property
    def nombre(self):
        return self._nombre

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if value < 0:
            pass  
        else:
            self._precio = value

    @property
    def tipo(self):
        return self._tipo

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if value < 0:
            pass  
        else:
            self._cantidad = value

    def valorTotal(self):
        return self._precio * self._cantidad

In [615]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [616]:
# Clase base Oferta
class Oferta:
    def __init__(self, descripcion="Sin descripción", codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos  # Lista de códigos específicos de productos
        self.tipos = tipos      # Lista de tipos de productos

    # Método para verificar si una oferta es aplicable a un producto y una cantidad dada
    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

# Subclase OfertaDescuento para aplicar descuentos en porcentaje
class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(*args, **kwargs)  # Aquí pasamos 'descripcion' a la clase base (si es necesario)
        self.descuento = descuento  # Descuento en porcentaje

    # Método para calcular el descuento
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            # El descuento se calcula sobre el precio total
            return (producto.precio * cantidad) * (self.descuento / 100)
        return 0

# Subclase Oferta2x1 para aplicar la promoción de "compra 2 y lleva 1 gratis"
class Oferta2x1(Oferta):
    def __init__(self, descripcion="Oferta 2x1", *args, **kwargs):
        super().__init__(descripcion, *args, **kwargs)  # Aquí pasamos 'descripcion' a la clase base (si es necesario)

    # Método para calcular el descuento según la promoción 2x1
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            free_items = cantidad // 2  # Por cada dos productos, uno es gratis
            return free_items * producto.precio
        return 0


In [617]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [618]:
# Clase Catalogo

import csv

class Catalogo:
    def __init__(self):
        self.productos = {}
        self.ofertas = []


    @classmethod
    def leer(cls, archivo):
        catalogo = cls()  # Creamos una nueva instancia de la clase Catalogo
        with open(archivo, mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                producto = Producto(
                    row['codigo'], row['nombre'], float(row['precio']),
                    row['tipo'], int(row['cantidad'])
                )
                catalogo.agregar(producto)  # Usamos la instancia para agregar productos
        return catalogo  # Devolvemos la instancia de Catalogo creada


    def guardar(self, archivo):
        with open(archivo, mode='w', newline='') as file:
            fieldnames = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
            writer = csv.DictWriter(file, fieldnames=fieldnames)

            writer.writeheader()  # Escribimos la cabecera
            for producto in self.productos.values():
                writer.writerow({
                    'codigo': producto.codigo,
                    'nombre': producto.nombre,
                    'precio': producto.precio,
                    'tipo': producto.tipo,
                    'cantidad': producto.cantidad
                })

    def agregar(self, *productos):
        for producto in productos:
            self.productos[producto.codigo] = producto

    def buscar(self, codigo):
        return self.productos.get(codigo, None)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscarOferta(producto, cantidad)
        if oferta:
            return oferta.calcularDescuento(producto, cantidad)
        return 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos.values())

    @property
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos.values())

    def informe(self):
        informe = f"Cantidad de productos: {self.cantidadProductos}\n"
        informe += f"Cantidad de unidades: {self.cantidadUnidades}\n"
        informe += f"Valor total: {self.valorTotal}\n"
        
        # Cálculo del precio promedio
        if self.cantidadUnidades > 0:
            precio_promedio = self.valorTotal / self.cantidadUnidades
        else:
            precio_promedio = 0  # Evitar división por cero si no hay unidades
        
        informe += f"Precio Promedio: {precio_promedio:.2f}\n"
        
        # Sección de Tipos de productos
        informe += "Tipos de productos: \n"
        tipos = sorted({p.tipo for p in self.productos.values()})  # Aseguramos que esté ordenado y sin duplicados
        for tipo in tipos:
            informe += f"{tipo}\n"
        
        # Sección de Ofertas
        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            informe += f"{oferta.descripcion}\n"
        
        return informe


    # Implementación del método vender
    def vender(self, producto, cantidad):
        """Reduce la cantidad de un producto del catálogo en base a una venta"""
        if producto.codigo in self.productos:
            if self.productos[producto.codigo].cantidad >= cantidad:
                self.productos[producto.codigo].cantidad -= cantidad
            else:
                raise ValueError("No hay suficiente stock para vender")
        else:
            raise ValueError("Producto no encontrado en el catálogo")



In [619]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [620]:
class Cliente: 
    def __init__(self, nombre, cuit): 
        self._nombre = nombre
        self._cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if not value:  # Rejects empty strings
            pass
        else:
            self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        # Verifying correct format XX-XXXXXXXX-X (13 characters)
        if len(value) == 13 and value[2] == '-' and value[11] == '-' and value[:2].isdigit() and value[3:11].isdigit() and value[12].isdigit():
            self._cuit = value
        else:
            pass  # Keeps previous value if invalid

In [621]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [622]:
class Factura:
    _ultimo_numero = 100  # Variable de clase para llevar el número de la última factura

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.productos = {}
        self.numero = self.nuevoNumero()
        self.subtotal = 0
        self.descuentos = 0
        self.total = 0

    @classmethod
    def ultimoNumero(cls, numero):
        cls._ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls._ultimo_numero += 1
        return cls._ultimo_numero

    # Método que establece el número de la última factura emitida
    @classmethod
    def ultimaFactura(cls, numero):
        cls._ultimo_numero = numero

    def agregar(self, producto, cantidad):
        if producto.codigo in self.productos:
            self.productos[producto.codigo]['cantidad'] += cantidad
        else:
            self.productos[producto.codigo] = {'producto': producto, 'cantidad': cantidad}

        self.subtotal += producto.precio * cantidad
        self.descuentos += self.catalogo.calcularDescuento(producto, cantidad)
        self.total = self.subtotal - self.descuentos
        self.catalogo.vender(producto, cantidad)

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(item['cantidad'] for item in self.productos.values())

    def imprimir(self):
        # Detalle inicial de la factura con el cliente
        detalle = f"Factura N°: {self.numero}\nCliente: {self.cliente.nombre} (CUIT: {self.cliente.cuit})\n\n"

        # Agregamos la lista de productos con el formato adecuado
        for item in self.productos.values():
            producto = item['producto']
            cantidad = item['cantidad']
            detalle += f"{cantidad} {producto.nombre} ({producto.tipo}) - {producto.precio:.2f} cada uno\n"

        # Agregamos los subtotales y totales
        detalle += f"\nSubtotal: {self.subtotal:.2f}\nDescuentos: {self.descuentos:.2f}\nTOTAL: {self.total:.2f}\n"

        # Agregamos los descuentos si los hay
        if self.descuentos > 0:
            for item in self.productos.values():
               producto = item['producto']
               cantidad = item['cantidad']
               oferta = self.catalogo.buscarOferta(producto, cantidad)
               if oferta:
                   if isinstance(oferta, OfertaDescuento):
                        # En lugar de verificar si el descuento es 10%, imprimimos el porcentaje de la oferta
                        detalle += f"Descuento {oferta.descuento}%\n"
                   elif isinstance(oferta, Oferta2x1):
                       detalle += "Oferta 2x1\n"

        return detalle



In [623]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")


Prueba pasada exitosamente!


In [624]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

AssertionError: 